In [201]:
import numpy as np
import random

## Defining paramters for the SGA

In [202]:
pop_size = 10
chromosome_length = 5
generation_number = 1
crossover_rate = 0.5
mutation_rate = 0.5
num_parents = 4

## Defining the Individual

In [203]:
class Individual:
    def __init__(self, genotype, age) -> None:
        self.genotype = genotype
        self.phenotype = int(genotype, 2)
        self.sin_fitness = self.calculate_sin_fitness()
        self.children = []
        self.age = age
    
    def calculate_sin_fitness(self):
        genotype_length = len(self.genotype)
        scaling_factor = (2**7) / (2**genotype_length)
        scaled_value = self.phenotype * scaling_factor
        
        # Scaling by 1 to avoid negative values
        return np.sin(scaled_value) + 1
    
    def add_children(self, children):
        self.children += children

    def grow_older(self):
        self.age += 1
    
    def __repr__(self):
        return f"Individual(genotype={self.genotype}, sin_fitness={self.sin_fitness})"

## Defining the population

In [204]:
population = []

## Sin Synthetic Problem

*a) Implement a function to generate an initial population for your genetic
algorithm.*

We are choosing to initialize the population, according to the size parameters. The actual genotypes (0's and 1's) will be generated randomly.

In [205]:
def init_population():
    for _ in range(pop_size):
        new_genotype = ""
        for _ in range(chromosome_length):
            new_genotype += str(random.randint(0,1))

        new_individual = Individual(new_genotype, age=0)
        population.append(new_individual)

In [206]:
init_population()
population

[Individual(genotype=11111, sin_fitness=0.004313013110820552),
 Individual(genotype=11101, sin_fitness=1.236661393364286),
 Individual(genotype=00111, sin_fitness=1.2709057883078692),
 Individual(genotype=01110, sin_fitness=0.47844899791308815),
 Individual(genotype=11001, sin_fitness=0.4936343588902412),
 Individual(genotype=00001, sin_fitness=0.2431975046920717),
 Individual(genotype=01011, sin_fitness=1.0177019251054136),
 Individual(genotype=10010, sin_fitness=1.2538233627620363),
 Individual(genotype=10100, sin_fitness=0.00611134607662478),
 Individual(genotype=01011, sin_fitness=1.0177019251054136)]

*b) Implement a parent selection function for your genetic algorithm. This function
should find the fittest individuals in the population, and select parents based
on this fitness.*

Here we are using the fitness function given by the sine function, as explained in the task. Moreover we are implementing a roulette-wheel approach as parent selection.

In [207]:
def roulette_wheel_selection(population):
    total_fitness = sum(individual.sin_fitness for individual in population)
    
    # Probability of choosing each individual based on the roulette wheel approach
    probabilities = [individual.sin_fitness/total_fitness for individual in population]

    # Choosing num_parents parents with the above probabilities for each individual
    parents = np.random.choice(population, size=num_parents, p=probabilities)
    
    return parents

In [208]:
parents = roulette_wheel_selection(population)
parents

array([Individual(genotype=00111, sin_fitness=1.2709057883078692),
       Individual(genotype=11001, sin_fitness=0.4936343588902412),
       Individual(genotype=00111, sin_fitness=1.2709057883078692),
       Individual(genotype=01110, sin_fitness=0.47844899791308815)],
      dtype=object)

*c) Implement a function that creates two offspring from two parents through
crossover. The offspring should also have a chance of getting a random
mutation.*

In [209]:
def crossover(parent1, parent2):
    gene_cutoff = random.randint(1, chromosome_length - 1)

    genotype_1 = parent1.genotype[: gene_cutoff] + parent2.genotype[gene_cutoff:]
    genotype_2 = parent2.genotype[: gene_cutoff] + parent1.genotype[gene_cutoff:]

    child_1 = Individual(mutate(genotype_1), age=0)
    child_2 = Individual(mutate(genotype_2), age=0)

    parent1.add_children([child_1, child_2])
    parent2.add_children([child_1, child_2])
        
def mutate(genotype):
    if np.random.random() < mutation_rate:
        char_list = list(genotype)

        random_index = np.random.randint(len(char_list))

        char_list[random_index] = '0' if char_list[random_index] == '1' else '1'

        return ''.join(char_list)
    else:
        return genotype

*d) Implement survivor selection.*

- For a 1-person team: Implement one survivor selection function that
selects the survivors of a population based on their fitness.
- For a 2-person team: Implement two such survivor selection functions.

Survivor selection function 1 chooses based on age. It sorts the population based on age, from youngest to oldest. Then outputs the youngest individuals.

In [ ]:
def age_selection(population):
    age_sorted_population = sorted(population, key=lambda x: x.age)
    return age_sorted_population[:pop_size]

Survivor selection function 2..